In [13]:
import os
import uproot
import ROOT as r
import math
import awkward as ak
import hist
from hist import Hist
from hist import loc
import sys
import argparse
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
sys.path.append('/sdf/group/hps/users/alspellm/projects/THESIS/analysis/reach_estimate')
from SimpEquations_validated import SimpEquations as simpeqs
import mpl_plot_utilities as mplutils
from matplotlib.backends.backend_pdf import PdfPages
import psutil
import glob
import re

In [14]:
indir = 'results_20230202_v2'
outdir = f'{indir}/results_summary'
outfilename = 'deltaZ_opt_results'
if not os.path.exists(outdir):
    os.mkdir(outdir)
    
def sort_key(file_path):
    # Extract the number following 'mass' using regular expressions
    match = re.search(r"mass_([\d.]+)", file_path)
    if match:
        # Convert the matched number to float for proper sorting
        return float(match.group(1))
    else:
        # If 'mass' number not found, return a large number to push it to the end
        return float('inf')
    
directory = f'/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/{indir}'
pattern = '*.root'
file_list = sorted(glob.glob(f"{directory}/{pattern}"), key=sort_key)
print(file_list)

outfile = r.TFile(f'{outdir}/{outfilename}.root', "RECREATE")

masses = []
best_nsigs = []
best_zbis = []
best_cutvals = []
best_zcuts = []
best_nbkgs = []

best_nsigs_right = []
best_zbis_right = []
best_cutvals_right = []
best_zcuts_right = []
best_nbkgs_right = []

best_nsigs_left = []
best_zbis_left = []
best_cutvals_left = []
best_zcuts_left = []
best_nbkgs_left = []

nsig_plots = {}
zbi_plots = {}
cutval_plots = {}
nbkg_plots = {}
zcut_plots = {}
fit_plots = {}

for filepath in file_list:
    filename = os.path.basename(filepath)
    print(filename)
    mass = filename.split('_')[1]
    
    
    infile = r.TFile(f'{directory}/{filename}','READ')
    infile.cd()
    
    keys = infile.GetListOfKeys()
    print(len(keys))
    if len(keys) == 0:
        break
    masses.append(float(mass))
    print('Mass:', mass)

    nsig_g = infile.Get('nsig_g')
    zbi_g = infile.Get('zbis_g')
    cutval_g = infile.Get('cut_values_g')
    zcut_g = infile.Get('zcuts_g')
    nbkg_g = infile.Get('nbkg_g')
    fit_p_g = infile.Get('bkg_fit_p_g')

    nsig_plots[f'mass_{mass}'] = nsig_g
    zbi_plots[f'mass_{mass}'] = zbi_g
    nbkg_plots[f'mass_{mass}'] = nbkg_g
    zcut_plots[f'mass_{mass}'] = zcut_g
    fit_plots[f'mass_{mass}'] = fit_p_g
    cutval_plots[f'mass_{mass}'] = cutval_g

    nsig_vals = np.array(nsig_g.GetY())
    zbi_vals = np.array(zbi_g.GetY())
    cutval_vals = np.array(cutval_g.GetY())
    zcut_vals = np.array(zcut_g.GetY())
    nbkg_vals = np.array(nbkg_g.GetY())
    fit_probs = np.array(fit_p_g.GetY())

    #Filter out bad background models
    bad_fit_idxs = np.where(fit_probs < 0.05)
    zbi_vals = np.delete(zbi_vals, bad_fit_idxs)
    nsig_vals = np.delete(nsig_vals, bad_fit_idxs)
    nbkg_vals = np.delete(nbkg_vals, bad_fit_idxs)
    zcut_vals = np.delete(zcut_vals, bad_fit_idxs)
    cutval_vals = np.delete(cutval_vals, bad_fit_idxs)


    #Get Best ZBi
    best_zbi_idx = np.argmax(zbi_vals)
    best_nsig = nsig_vals[best_zbi_idx]
    best_nbkg = nbkg_vals[best_zbi_idx]
    best_zcut = zcut_vals[best_zbi_idx]
    best_zbi = zbi_vals[best_zbi_idx]
    best_cutval = cutval_vals[best_zbi_idx]

    best_nsigs.append(best_nsig)
    best_nbkgs.append(best_nbkg)
    best_zbis.append(best_zbi)
    best_zcuts.append(best_zcut)
    best_cutvals.append(best_cutval)

    nsig_90p_idxs = np.where(abs(nsig_vals) > abs(0.9*best_nsig))[0]
    #print('left:', zbi_90p_idxs[:int(np.where(zbi_90p_idxs == best_zbi_idx)[0])])
    try:
        left_index = nsig_90p_idxs[:int(np.where(nsig_90p_idxs == best_zbi_idx)[0])][-1]
    except:
        left_index = best_zbi_idx
    
    try:
        right_index = nsig_90p_idxs[int(np.where(nsig_90p_idxs == best_zbi_idx)[0])+1:][0]
    except:
        right_index = best_zbi_idx

    #last index above 
    best_nsigs_right.append(nsig_vals[right_index])
    best_zbis_right.append(zbi_vals[right_index])
    best_cutvals_right.append(cutval_vals[right_index])
    best_zcuts_right.append(zcut_vals[right_index])
    best_nbkgs_right.append(nbkg_vals[right_index])

    #first index above
    best_nsigs_left.append(nsig_vals[left_index])
    best_zbis_left.append(zbi_vals[left_index])
    best_cutvals_left.append(cutval_vals[left_index])
    best_zcuts_left.append(zcut_vals[left_index])
    best_nbkgs_left.append(nbkg_vals[left_index])
       

 
outfile.cd()

#Nsig
c = r.TCanvas('expected_signal_deltaZ','expected_signal_deltaZ', 2400, 1400)
c.cd()
nsig_g = r.TGraph(len(masses), np.array(masses), np.array(best_nsigs))
nsig_g.SetName('expected_signal_g')
nsig_g.SetTitle('Expected Signal deltaZ Max ZBi; VD Mass [GeV]; Tenpct Expected Signalx5')
nsig_g.Draw()
nsig_g.Write()
c.Write()
c.SaveAs(f'{outdir}/{c.GetName()}.png')

#ZBi
c = r.TCanvas('max_ZBi_deltaZ','max_ZBi_deltaZ', 2400, 1400)
c.cd()
zbi_g = r.TGraph(len(masses), np.array(masses), np.array(best_zbis))
zbi_g.SetName('max_ZBi_g')
zbi_g.SetTitle('deltaZ Max ZBi; VD Mass [GeV]; Tennpct Signalx5 Max ZBi')
zbi_g.Draw()
zbi_g.Write()
c.Write()
c.SaveAs(f'{outdir}/{c.GetName()}.png')

#nbkgs
c = r.TCanvas('nbkg_deltaZ','nbkg_deltaZ', 2400, 1400)
c.cd()
nbkg_g = r.TGraph(len(masses), np.array(masses), np.array(best_nbkgs))
nbkg_g.SetName('nbkg_g')
nbkg_g.SetTitle('Nbkg deltaZ Max ZBi; VD Mass [GeV]; Nbkg')
nbkg_g.Draw()
nbkg_g.Write()
c.Write()
c.SaveAs(f'{outdir}/{c.GetName()}.png')

#Zcut
c = r.TCanvas('zcut_deltaZ','zcut_deltaZ', 2400, 1400)
c.cd()
zbi_g = r.TGraph(len(masses), np.array(masses), np.array(best_zcuts))
zbi_g.SetName('zcut_deltaZ_g')
zbi_g.SetTitle('Zcut deltaZ Max ZBi; VD Mass [GeV]; Zcut [mm]')
zbi_g.Draw()
zbi_g.Write()
c.Write()
c.SaveAs(f'{outdir}/{c.GetName()}.png')

#cutvalues
c = r.TCanvas('deltaZ_cut','deltaZ_cut', 2400, 1400)
c.cd()
cutval_g = r.TGraph(len(masses), np.array(masses), np.array(best_cutvals))
cutval_g.SetName('deltaZ_cut_g')
cutval_g.SetTitle('deltaZ Cut Value Max ZBi; VD Mass [GeV]; deltaZ [mm]')
cutval_g.Draw()
cutval_g.Write()
c.Write()
c.SaveAs(f'{outdir}/{c.GetName()}.png')
  

['/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/results_20230202_v2/mass_40.0_deltaz_opt.root', '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/results_20230202_v2/mass_45.0_deltaz_opt.root', '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/results_20230202_v2/mass_50.0_deltaz_opt.root', '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/results_20230202_v2/mass_55.0_deltaz_opt.root', '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/results_20230202_v2/mass_60.0_deltaz_opt.root', '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/results_20230202_v2/mass_65.0_deltaz_opt.root', '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/results_20230202_v2/mass_70.0_deltaz_opt.root', '/sdf/group/hps/users/alspellm/projects/THESIS/analysis/deltaZ/optimization/results_20230202_v2/mass_75.0_deltaz_opt.root', '/sdf/g

Info in <TCanvas::Print>: png file results_20230202_v2/results_summary/expected_signal_deltaZ.png has been created
Info in <TCanvas::Print>: png file results_20230202_v2/results_summary/max_ZBi_deltaZ.png has been created
Info in <TCanvas::Print>: png file results_20230202_v2/results_summary/nbkg_deltaZ.png has been created
Info in <TCanvas::Print>: png file results_20230202_v2/results_summary/zcut_deltaZ.png has been created
Info in <TCanvas::Print>: png file results_20230202_v2/results_summary/deltaZ_cut.png has been created


In [15]:
  
    
#TgraphErros with 0.9*ZbiMax Interval
#Nsig
xerrors_l = np.zeros(len(masses))
xerrors_h = np.zeros(len(masses))

c = r.TCanvas('deltaZ_cut_values_90p','deltaZ_cut_values_90p', 2400, 1400)
c.cd()
cutval_hl_g = r.TGraphAsymmErrors(len(masses), np.array(masses), np.array(best_cutvals), xerrors_l, xerrors_h, np.array(best_cutvals)-np.array(best_cutvals_right), np.array(best_cutvals_left)-np.array(best_cutvals))
cutval_hl_g.SetName('deltaZ_cut_values_90p_g')
cutval_hl_g.SetTitle('deltaZ Cut Values 90pct Max ZBi; VD Mass [GeV]; deltaZ [mm]')
cutval_hl_g.Draw()
linear_func = r.TF1("linear_func", "[0] + [1]*x", 40.0, 130.0)
cutval_hl_g.Fit(linear_func,"","",40.0,130.0)
cutval_hl_g.Write()
c.Write()
c.SaveAs(f'{outdir}/{c.GetName()}.png')

 FCN=779.609 FROM MIGRAD    STATUS=CONVERGED      50 CALLS          51 TOTAL
                     EDM=3.35058e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.29382e+01   2.29798e-01   1.01638e-03  -6.03386e-03
   2  p1          -1.01124e-03   2.56170e-03   1.13302e-05  -2.44973e-01


Warning in <TCanvas::Constructor>: Deleting canvas with same name: deltaZ_cut_values_90p
Info in <TCanvas::Print>: png file results_20230202_v2/results_summary/deltaZ_cut_values_90p.png has been created


In [16]:
#Plot all masses together
for key, g in nsig_plots.items():
    g.SetName(f'nsig_{key}')
    g.SetTitle(f'nsig_{key}')
    g.Write()

for key, g in  zbi_plots.items():
    g.SetName(f'zbi_{key}')
    g.SetTitle(f'zbi_{key}')
    g.Write()
for key, g in  zcut_plots.items():
    g.SetName(f'zcut_{key}')
    g.SetTitle(f'zcut_{key}')
    g.Write()
for key, g in  nbkg_plots.items():
    g.SetName(f'nbkg_{key}')
    g.SetTitle(f'nbkg_{key}')
    g.Write()
for key, g in cutval_plots.items():
    g.SetName(f'cutval_{key}')
    g.SetTitle(f'cutval_{key}')
    g.Write()
for key, g in  fit_plots.items():
    g.SetName(f'fit_{key}')
    g.SetTitle(f'fit_{key}')
    g.Write()
    
outfile.Close()